# CommunityResource creation monitoring

- uses the public API
- uses a file to store the latest CommunityResource we've seen in between cron runs
- notifies on Slack (webhook)

In [ ]:
import requests, os
from datetime import datetime, timedelta
from pathlib import Path

In [ ]:
# in minutes - /!\ set the cron to the same frequency
FREQUENCY = 10
wh_url = os.environ['NB_SLACK_WEBHOOK_URL_ACTIVITY']

In [ ]:
last_run = datetime.now() - timedelta(minutes=FREQUENCY)

**send a message to Slack API**

In [ ]:
def send_message(msg):
    print(msg)
    r = requests.post(wh_url, json={'text': msg})
    assert r.text == 'ok'

In [ ]:
def notify_cr(cr):
    dataset_link = f"<{cr['dataset']['page']}|{cr['dataset']['title']}>" if cr['dataset'] else '*None*'
    send_message(f":panda_face: new _CommunityResource_ on {dataset_link}: {cr['url']}")

In [ ]:
cr_url = 'https://www.data.gouv.fr/api/1/datasets/community_resources/?sort=-created_at'
r = requests.get(cr_url)
new_last_found = None
for cr in r.json()['data']:
    cr_date = datetime.fromisoformat(cr['created_at'])
    if last_run and cr_date < last_run:
        break
    notify_cr(cr)